In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter  # TensorBoard-hoz
import torch.nn.functional as F
# from ann_visualizer.visualize import ann_viz
# import visualkeras
import tensorflow as tf
# from torchviz import make_dot
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Elérési útvonalak
data_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/"
train_dir = data_dir + "train/"
val_dir = data_dir + "val/"
test_dir = data_dir + "test/"

# Transzformációs paraméterek
image_size = (224, 224)
batch_size = 112

# Kép adatgenerátor létrehozása
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train Dataset létrehozása
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'  # Mivel két osztály van: "PNEUMONIA" és "NORMAL"
)

# Validation Dataset létrehozása
val_dataset = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Test Dataset létrehozása
test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [3]:
from tensorflow.keras import layers, models

# Modell létrehozása
from tensorflow.keras import layers, models

model = models.Sequential()

# Block 1
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Flatten and fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='sigmoid'))
model.add(layers.Dense(4096, activation='sigmoid'))


# Output layer with Sigmoid activation for binary classification
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [4]:
# Download ngrok
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

# Start TensorBoard in the background
# get_ipython().system_raw(f'tensorboard --logdir /kaggle/working/ --host 0.0.0.0 --port 6006 &')

# Get the public URL using ngrok
# get_ipython().system_raw('./ngrok http 6007 &')

# Import the time module
# import time
# time.sleep(1.0)

# Print the public URL
# !curl -s http://localhost:4040/api/tunnels | python3 -c 'import sys, json; print("TensorBoard will be available at the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'


In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Tanulási ráta dinamikussá tétele a teljesítmény alapján
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=3, min_lr=1e-6)

# Modell fordítása (compile)
model.compile(
    optimizer=Adam(
        learning_rate=0.002,  # Tanulási ráta
        beta_1=0.5,  # Exponenciális mozgóátlag decay faktora az első momentumra
        beta_2=0.999,  # Exponenciális mozgóátlag decay faktora a második momentumra
        epsilon=1e-07,  # Az osztóhoz hozzáadott kis konstans
        amsgrad=False,  # Amsgrad algoritmus alkalmazása
        # weight_decay=None,  # Súlycsökkentés (L2 súlycsökkentés) - nem támogatott
        clipnorm=None,  # Szabályozza a súlyvektornormákat
        clipvalue=None,  # Szabályozza a súlyértékeket
        global_clipnorm=None,  # Szabályozza az összes súlyvektornormát
        use_ema=False,  # Exponenciális mozgóátlag használata a súlyok frissítéséhez
        ema_momentum=0.8,  # Az exponenciális mozgóátlag momentuma
        ema_overwrite_frequency=None,  # Az exponenciális mozgóátlag súlyainak felülírásának gyakorisága
        jit_compile=True,  # Just-In-Time fordítás (JIT) alkalmazása
        name='Adam'  # Az optimizer neve
    ),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ModelCheckpoint callback létrehozása a legjobb súlyok mentéséhez
checkpoint = ModelCheckpoint("best_weights", save_best_only=True)

# Modell tanítása
num_epochs_fine_tune = 9999

# Train
history_fine_tune = model.fit(
    train_dataset,
    epochs=num_epochs_fine_tune,
    validation_data=val_dataset,
    callbacks=[checkpoint, reduce_lr],
    verbose=1                        
)

# Metrikák kinyerése a history objektumból
train_loss = history_fine_tune.history['loss']
train_accuracy = history_fine_tune.history['accuracy']
val_loss = history_fine_tune.history['val_loss']
val_accuracy = history_fine_tune.history['val_accuracy']


Epoch 1/9999


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/max_pooling2d_2/MaxPool' defined at (most recent call last):
    File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    File